In [2]:
import os
import torch
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import ultralytics
import time
import imutils

In [3]:
pathtoimage = "dataset_modelo"
resultspath = "DS_letras_cut"
if not os.path.exists(resultspath):
    os.makedirs(resultspath)

def generate_boxes_confidences_classids_v8(outs, threshold):
		boxes = []
		confidences = []
		classids = []

		for out in outs:
				for box in out.boxes:
					x1, y1, x2, y2 = [round(x) for x in box.xyxy[0].tolist()]
					class_id = box.cls[0].item()
					prob = round(box.conf[0].item(), 2)
					if prob > threshold:
						# Append to list
						boxes.append([x1, y1, x2-x1, y2-y1])
						confidences.append(float(prob))
						classids.append(class_id)
	
		return boxes, confidences, classids

def yolov8_warmup(model, repetitions=1, verbose=False):
    # Warmup model
    startTime = time.time()
    # create an empty frame to warmup the model
    for i in range(repetitions):
        warmupFrame = np.zeros((360, 640, 3), dtype=np.uint8)
        model.predict(source=warmupFrame, verbose=verbose)
    print(f"Model warmed up in {time.time() - startTime} seconds")

print("Loading model...")

#prevtime = time.time()
model = ultralytics.YOLO("/home/jabv/Desktop/larc30000_yolov87/weights/letras_colores_v2.pt")
yolov8_warmup(model, 10, False)
#print(f"Model loaded in {time.time() - prevtime} seconds")


import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor
sam_model = "h"

#wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
if sam_model =="h":
  sam_checkpoint = "sam_vit_h_4b8939.pth"
  model_type = "vit_h"
else:
  sam_checkpoint = "sam_vit_l_0b3195.pth"
  model_type = "vit_l"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

# Creating annotation in COCO format
#{"id": 0, "file_name": "0.jpg", "height": 480, "width": 736}
images=[]
annotations=[]
categories=[]

img_id=0
anno_id=0

#check if results directory exists, else create it
if not os.path.exists(resultspath):
  os.makedirs(resultspath)


imgPaths = os.listdir(pathtoimage)
print(imgPaths)

i=0

for imgPath in imgPaths:
    print(f"Processing image: {imgPath}")
    img = imutils.resize(cv2.imread(f"{pathtoimage}/{imgPath}"))
    if img is None:
        continue
    #results = model(img)
    results = model(img, verbose=False)
    boxes, confidences, classids = generate_boxes_confidences_classids_v8(results, 0.85)
    for i in range(len(boxes)):
        x,y,w,h = boxes[i]
        class_name = model.model.names[classids[i]]
        ran_sam = False

        sam_bounding_box = (np.array([x, y, x+w, y+h]))

        #run sam
        if ran_sam == False:
            predictor.set_image(img)
            ran_sam = True

        mask, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=sam_bounding_box,
            multimask_output=False,
        )

        contours, _ = cv2.findContours(mask[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Your call to find the contours
        # threshold input image using otsu thresholding as mask and refine with morphology
        ret, pngmask = cv2.threshold(mask[0].astype(np.uint8), 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
        kernel = np.ones((9,9), np.uint8)
        pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_CLOSE, kernel)
        pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_OPEN, kernel)

        # put mask into alpha channel of result
        result = img.copy()
        result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
        result[:, :, 3] = pngmask
        #cv2.imwrite(f"{resultspath}/{cutobject}_{i}.png", result)
        #to save with same name as original file
        # if already exists, save with _1, _2, etc

        if not os.path.exists(f"{resultspath}/{class_name}"):
            os.mkdir(f"{resultspath}/{class_name}")

        if os.path.exists(f"{resultspath}/{class_name}/{imgPath[:-4]}.png"):
            if os.path.exists(f"{resultspath}/{class_name}/{imgPath[:-4]}_1.png"):
                print("File already exists, saving with _2")
                cv2.imwrite(f"{resultspath}/{class_name}/{imgPath[:-4]}_2.png", result)
            print("File already exists, saving with _1")
            temppath = f"{resultspath}/{class_name}/{imgPath[:-4]}_1.png"
            print(temppath)
            cv2.imwrite(f"{resultspath}/{class_name}/{imgPath[:-4]}_1.png", result)

        cv2.imwrite(f"{resultspath}/{class_name}/{imgPath[:-4]}.png", result)
        i=i+1
        ran_sam = False


Loading model...
Model warmed up in 1.7003874778747559 seconds
['IMG_0928.png', 'IMG_3348.png', 'IMG_1485.png', 'IMG_1419.png', 'IMG_1190.png', 'IMG_1811.png', 'IMG_1917.png', 'IMG_0721.png', 'IMG_0678.png', 'IMG_1608.png', 'IMG_1031.png', 'IMG_3323.png', 'IMG_2319.png', 'IMG_1435.png', 'IMG_2535.png', 'IMG_2148.png', 'IMG_2045.png', 'IMG_1754.png', 'IMG_2216.png', 'IMG_2048.png', 'IMG_1161.png', 'IMG_2519.png', 'IMG_3315.png', 'IMG_2119.png', 'IMG_2214.png', 'IMG_1371.png', 'IMG_1886.png', 'IMG_1249.png', 'IMG_0790.png', 'IMG_0898.png', 'IMG_3285.png', 'IMG_2491.png', 'IMG_2454.png', 'IMG_0692.png', 'IMG_1759.png', 'IMG_1680.png', 'IMG_2059.png', 'IMG_1572.png', 'IMG_2616.png', 'IMG_1660.png', 'IMG_2054.png', 'IMG_2361.png', 'IMG_2138.png', 'IMG_2287.png', 'IMG_1467.png', 'IMG_1603.png', 'IMG_1774.png', 'IMG_2095.png', 'IMG_1464.png', 'IMG_2154.png', 'IMG_2466.png', 'IMG_2174.png', 'IMG_0764.png', 'IMG_2434.png', 'IMG_1495.png', 'IMG_0700.png', 'IMG_2160.png', 'IMG_1098.png', 'IMG_170

In [4]:
from PIL import Image
import os

datasetpath = "DS_letras_cut/"
resultspath = "DS_letras/"

#create a result folder if it doesn't exist
if not os.path.exists(resultspath):
    os.makedirs(resultspath)
    os.makedirs(resultspath+"A/")
    os.makedirs(resultspath+"B/")
    os.makedirs(resultspath+"C/")
    os.makedirs(resultspath+"D/")
    os.makedirs(resultspath+"E/")
    os.makedirs(resultspath+"F/")
    os.makedirs(resultspath+"G/")
    os.makedirs(resultspath+"H/")
    os.makedirs(resultspath+"I/")
    os.makedirs(resultspath+"verde/")
    os.makedirs(resultspath+"amarillo/")
    os.makedirs(resultspath+"azul/")
    os.makedirs(resultspath+"rojo/")

fg_folders = [
    ("A/"),
    ("B/"),
    ("C/"),
    ("D/"),
    ("E/"),
    ("F/"),
    ("G/"),
    ("H/"),
    ("I/"),
    ("verde/"),
    ("amarillo/"),
    ("azul/"),
    ("rojo/")
]

for folder in fg_folders:
    for filename in os.listdir(f"{datasetpath}{folder}"):
        try:
            print(f"{filename} started")
            myImage = Image.open(datasetpath+folder+filename)
            black = Image.new('RGBA', myImage.size)
            myImage = Image.composite(myImage, black, myImage)
            print("aqui")
            myCroppedImage = myImage.crop(myImage.getbbox())
            myCroppedImage.save(f"{resultspath}{folder}{filename}")
            print(f"{filename} done")
        except:
            print(f"{filename} failed")
            continue
print("all done")

IMG_0721.png started
aqui
IMG_0721.png failed
IMG_0678.png started
aqui
IMG_0678.png failed
IMG_3323.png started
aqui
IMG_3323.png failed
IMG_3315.png started
aqui
IMG_3315.png failed
IMG_0790.png started
aqui
IMG_0790.png failed
IMG_0692.png started
aqui
IMG_0692.png failed
IMG_0764.png started
aqui
IMG_0764.png failed
IMG_0700.png started
aqui
IMG_0700.png failed
IMG_0638.png started
aqui
IMG_0638.png failed
IMG_0743.png started
aqui
IMG_0743.png failed
IMG_3342.png started
aqui
IMG_3342.png failed
IMG_3313.png started
aqui
IMG_3313.png failed
IMG_0603.png started
aqui
IMG_0603.png failed
IMG_0699.png started
aqui
IMG_0699.png failed
IMG_0679.png started
aqui
IMG_0679.png failed
IMG_0673.png started
aqui
IMG_0673.png failed
IMG_0781.png started
aqui
IMG_0781.png failed
IMG_0768.png started
aqui
IMG_0768.png failed
IMG_0612.png started
aqui
IMG_0612.png failed
IMG_0604.png started
aqui
IMG_0604.png failed
IMG_3353.png started
aqui
IMG_3353.png failed
IMG_0737.png started
aqui
IMG_0737